### Import Libraries

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers
from keras import activations, initializers, constraints
from keras.regularizers import l2
import os
import pandas as pd
from numpy import nan_to_num, array, empty_like, empty, vstack, concatenate, linspace, tile
from keras.layers import GlobalAveragePooling2D,Conv1D, Permute, DepthwiseConv2D,ELU, AveragePooling2D, Conv2D, MaxPooling2D, ThresholdedReLU, UpSampling2D, SeparableConv2D, Conv2DTranspose, Lambda, Activation, Input, PReLU, Add,BatchNormalization, Multiply
from keras.layers import Dense, Dropout, Flatten
from scipy.signal import filtfilt, butter
import matplotlib.pyplot as plt
import pickle
from keras import backend as K
from keras.models import Model
import gzip
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import layers
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]
import numpy, scipy.io
from keras.engine import training
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.models import load_model
import tensorflow as tf
import h5py
from keras import backend as K
import os
import numpy as np
import scipy
from keras.callbacks import Callback
from keras.models import Model
from keras.layers import Lambda
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from scipy import io
from sklearn.model_selection import KFold, StratifiedKFold


In [ ]:
def graph_conv_op(x, num_filters, graph_conv_filters, kernel):

    if len(x.get_shape()) == 2:
        conv_op = K.dot(graph_conv_filters, x)
        conv_op = tf.split(conv_op, num_filters, axis=0)
        conv_op = K.concatenate(conv_op, axis=1)
    elif len(x.get_shape()) == 3:
        conv_op = K.batch_dot(graph_conv_filters,x)
        conv_op = tf.split(conv_op, num_filters, axis=1)
        conv_op = K.concatenate(conv_op, axis=2)
    else:
        raise ValueError('x must be either 2 or 3 dimension tensor'
                         'Got input shape: ' + str(x.get_shape()))

    conv_out = K.dot(conv_op, kernel)
    return conv_out

class GraphCNN(layers.Layer):

    def __init__(self,
                 output_dim,
                 num_filters,
                 graph_conv_filters,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(GraphCNN, self).__init__(**kwargs)

        self.output_dim = output_dim
        self.num_filters = num_filters
        if num_filters != int(graph_conv_filters.get_shape().as_list()[-2]/graph_conv_filters.get_shape().as_list()[-1]):
            raise ValueError('num_filters does not match with graph_conv_filters dimensions.')
        self.graph_conv_filters = graph_conv_filters

        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_initializer.__name__ = kernel_initializer
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

    def build(self, input_shape):

        self.input_dim = input_shape[-1]
        kernel_shape = (self.num_filters * self.input_dim, self.output_dim)

        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.output_dim,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True

    def call(self, input):

        output = graph_conv_op(input, self.num_filters, self.graph_conv_filters, self.kernel)
        if self.use_bias:
            output = K.bias_add(output, self.bias)
        if self.activation is not None:
            output = self.activation(output)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = (input_shape[0], self.output_dim)
        return output_shape

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'num_filters': self.num_filters,
            'graph_conv_filters': self.graph_conv_filters,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer': regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(GraphCNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



In [ ]:

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("tensorflow")

# Hyperparameter
Chans = 8
Samples = 312
f = 5
kf = KFold(n_splits=f,shuffle = True)
hidden_units = [16, 16]
learning_rate = 0.001
dropout_rate = 0.25
epochs = 25
batch_size = 64
out1 = 8
num_filters = 2
nb_classes = 3


### Model

## We use this Function for preparing the node representation

def create_ffn(hidden_units, dropout_rate, name=None):
    fnn_layers = []

    for units in hidden_units:
        fnn_layers.append(layers.BatchNormalization())
        fnn_layers.append(layers.Dropout(dropout_rate))
        fnn_layers.append(layers.Dense(units, activation=tf.nn.elu))

    return keras.Sequential(fnn_layers, name=name)


A = scipy.io.loadmat('/content/drive/MyDrive/Data/Adj_Location.mat')['adj']
#A = A[0:3,0:3]
graph_conv_filters = np.concatenate([A, np.matmul(A, A)], axis=0)
graph_conv_filters = np.expand_dims(graph_conv_filters,axis = 0)

a = graph_conv_filters

for i in range(batch_size-1):
  a = np.concatenate((a,graph_conv_filters),axis = 0)
graph_conv_filters = K.constant(a)
print(graph_conv_filters.shape)
#Changing to Float 32
K.set_floatx('float32')

input1   =  Input(shape = (Chans, Samples))

block1 = create_ffn(hidden_units,dropout_rate,name = 'Preprocess')(input1)

block2 = GraphCNN(out1, num_filters, graph_conv_filters, activation='elu',
                       kernel_regularizer=l2(5e-4))(block1)
block3 = tf.concat((block1,block2),axis = -1)

block4 = GraphCNN(out1, num_filters, graph_conv_filters, activation='elu',
                       kernel_regularizer=l2(5e-4))(block3)

block5 = tf.concat((block3,block4),axis = -1)

block6 = create_ffn(hidden_units,dropout_rate,name = 'Postprocess')(block5)

flatten      = Flatten(name = 'flatten')(block6)

dense        = Dense(nb_classes, name = 'dense')(flatten)

softmax      = Activation('softmax', name = 'softmax')(dense)


model = Model(inputs = input1, outputs = softmax)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipvalue=0.1),

              metrics=['mse'])

model.summary()

(64, 16, 8)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8, 312)]     0           []                               
                                                                                                  
 Preprocess (Sequential)        (None, 8, 16)        6592        ['input_2[0][0]']                
                                                                                                  
 graph_cnn_2 (GraphCNN)         (64, 8, 8)           264         ['Preprocess[0][0]']             
                                                                                                  
 tf.concat_2 (TFOpLambda)       (64, 8, 24)          0           ['Preprocess[0][0]',             
                                                                  'graph_cnn_2[0][0]']      

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#subject_path = ['/content/drive/MyDrive/Data/subj1_July28.mat','/content/drive/MyDrive/Data/subj2_July28.mat','/content/drive/MyDrive/Data/subj12_July28.mat']
subject_path = ['/content/drive/MyDrive/Data/sub1_Aug15_ccekh.mat','/content/drive/MyDrive/Data/sub2_Aug15_ccekh.mat','/content/drive/MyDrive/Data/sub12_Aug15_ccekh.mat']


accuracies = []


for i in range(len(subject_path)):
  a = scipy.io.loadmat(subject_path[i])['Sig']
  signal = []
  for ke in range(a.shape[-1]):
    signal.append(a[:,:,ke])
  signal = np.array(signal)
  signal = np.transpose(signal,(0,2,1))
  signal = np.expand_dims(signal,axis=-1)
  label = np.zeros((signal.shape[0],len(subject_path)))
  if i==0:
    F_signal = signal
    label[:,i]=1
    F_label = label
  else:
    F_signal=np.concatenate((F_signal,signal),axis=0)
    if i==1:
     label[:,i]=1
     F_label=np.concatenate((F_label,label),axis=0)
    if i==2:
     label[:,i]=1
     F_label=np.concatenate((F_label,label),axis=0)


print(F_signal.shape) #288x3x256x1,,,234x8x312x1
print(F_label.shape) #288x3,,, 234x3

Acc = []
pretrained = 0
for k in range(0,epochs):
    if pretrained ==1:
      model = tf.keras.models.load_model('/content/drive/MyDrive/Data/weights-new-improvement-Graph-'+str(k-1)+'.hdf5')
      pretrained = 0
      print('Model is loaded')
    if (k)%30==0 and k!=0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr*.1)
        print("lr changed to {}".format(lr*.1))
    accuracies = []
    for train_index, test_index in kf.split(F_signal):
      # Split the data into train and test sets based on the fold indices
      X_train, X_test = F_signal[train_index], F_signal[test_index]
      y_train, y_test = F_label[train_index], F_label[test_index]
      for m in range(250):

          trainin_it=np.empty((batch_size,8,312,1))
          trainout_it=np.empty((batch_size,len(subject_path)))

          for bac in range(batch_size):

              rand_l=np.random.randint(X_train.shape[0])


              trainin_it[bac,:,:,0]= X_train[rand_l,:,:,0]
              trainout_it[bac,:]= y_train[rand_l,:]

          trainin_it = tf.squeeze(trainin_it)

          model.fit(trainin_it, trainout_it,
                    batch_size=batch_size,
                    epochs=1,
                    verbose=1)

      trainin_it=[]
      trainout_it=[]


      #### Validation

      trainin_val=np.empty((batch_size,8,312,1))
      trainout_val=np.empty((batch_size,len(subject_path)))
      correct = 0
      total = 0
      for bac in range(batch_size): #for bac in range(len(X_test)):

          rand_l=np.random.randint(X_test.shape[0])


          trainin_val[bac,:,:,0]= X_test[rand_l,:,:,0]
          trainout_val[bac,:]= y_test[rand_l,:]

      trainin_val = tf.squeeze(trainin_val)
      print(trainin_val.shape)
      Prediction = model.predict(trainin_val,batch_size=batch_size)
      for i in range(len(Prediction)):
          act_label = numpy.argmax(trainout_val[i]) # act_label = 1 (index)
          pred_label = numpy.argmax(Prediction[i]) # pred_label = 1 (index)
          if(act_label == pred_label):
              correct += 1
          total += 1
      accuracy = (correct/total)
      accuracies.append(accuracy)

      # Calculate the average accuracy across all folds
    average_accuracy = sum(accuracies) / f
    Acc.append(average_accuracy)
    print('Average Acuuracy over %d fold is %.2f'%(f,average_accuracy))
    with open('//content/drive/MyDrive/Data/Avg-Graph', 'wb') as fp:
      pickle.dump(Acc, fp)
    #model.save('/content/drive/MyDrive/Data/weights-new-improvement-Graph-'+str(k)+'.hdf5')
    print('Model is saved')

    print("The epoch is",k+1)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 24ms/step - loss: 0.2045 - mse: 0.0357
(64, 8, 312)
1/1 [==============================] - 0s 17ms/step - loss: 0.2001 - mse: 0.0405
(64, 8, 312)
1/1 [==============================] - 0s 392ms/step - loss: 0.1789 - mse: 0.0305
(64, 8, 312)
1/1 [==============================] - 0s 33ms/step
Average Acuuracy over 5 fold is 0.98
Model is saved
The epoch is 10
1/1 [==============================] - 0s 23ms/step - loss: 0.2059 - mse: 0.0343
(64, 8, 312)
1/1 [==============================] - 0s 16ms/step - loss: 0.1985 - mse: 0.0372
(64, 8, 312)
1/1 [==============================] - 0s 226ms/step - loss: 0.2927 - mse: 0.0550
(64, 8, 312)
1/1 [==============================] - 0s 26ms/step - loss: 0.2551 - mse: 0.0466
(64, 8, 312)
1/1 [==============================] - 0s 21ms/step - loss: 0.2029 - mse: 0.0391
(64, 8, 312)
1/1 [==============================] - 0s 36ms/step
Average Acuuracy over 5

KeyboardInterrupt: ignored

In [ ]:
with open('/content/drive/MyDrive/Data/Avg-Graph','rb') as f:
  x = pickle.load(f)
x

[0.6,
 0.68125,
 0.803125,
 0.85,
 0.921875,
 0.928125,
 0.978125,
 0.95625,
 0.975,
 0.98125,
 0.975,
 0.996875,
 1.0]

In [ ]:
Prediction = model.predict(trainin_val)

1/1 [==============================] - 0s 24ms/step


In [ ]:
aa = np.random.randn(64,8,312)
bb = np.random.randn(64,16,8)

c = K.batch_dot(bb,aa)
c = tf.split(c,2,axis = 1)
c = K.concatenate(c, axis=2)
c = np.array(c)
input1   =  Input(shape = (Chans, Samples))

block1 = create_ffn(hidden_units,dropout_rate,name = 'Preprocess')(input1)

block2 = GraphCNN(out1, num_filters, graph_conv_filters, activation='elu',
                       kernel_regularizer=l2(5e-4))(block1)

block3 = block1+block2

block4 = GraphCNN(out1, num_filters, graph_conv_filters, activation='elu',
                       kernel_regularizer=l2(5e-4))(block3)

block5 = block3+block4

block6 = create_ffn(hidden_units,dropout_rate,name = 'Postprocess')(block5)

model = Model(inputs = input1, outputs = block6)
p = model.predict(aa)
p.shape

2/2 [==============================] - 1s 7ms/step


(64, 8, 32)